# ParkingEnv Interactive Demo

This notebook lets you experiment with the modular parking environment without leaving Jupyter.

基础运行参数：

- 设置演示时常用的控制变量（轮次数、模式、最大步数、动画速度）。

- 修改这些值后再次运行即可影响后续 CLI 启动的行为。

## Manual / Random Runs (Qt Window Only)

Set the parameters below and run the launcher cell—the demo will open in a Qt window. 

For random policy runs, set `mode = "random"` and adjust `sleep_scale` if you want to slow things down.

Keyboard controls in manual mode: `↑/↓` throttle, `←/→` steering, `Esc` exit.


In [730]:
from pathlib import Path

episodes = 1
max_steps = 4000
sleep_scale = 0.5    # Larger values slow the animation

mode = "manual"  # "manual" or "random"
# mode = "random"

## 生成随机训练配置 (随机地图，小车大小等等参数)

- 通过调用 CLI 工具快速输出一份随机配置，覆盖 `generated_configs/train_001.json`。(也可以手动修改)

- 在执行前请确认已激活 `parking-rl` 环境并安装依赖，否则命令会报错。


In [731]:
# !pip install -r requirements.txt
# !conda activate parking-rl
# !python generate_training_config.py --out generated_configs/train_001.json
# !python generate_training_config.py --out generated_configs/notebook_override.json

## 指定配置文件路径

- 控制是否写入 Notebook 生成的覆盖文件，以及 CLI 将加载哪份 JSON。

- 默认指向 `generated_configs/notebook_override.json`，需要切换时可改为训练集或新生成的配置。


In [732]:
write_notebook_config = False  # Set to False to keep existing file contents
config_path = Path("generated_configs/notebook_override.json")
# config_path = Path("generated_configs/train_001.json")

## 加载默认配置副本

- 读取 `build_config()` 的默认度数配置并深拷贝，方便在 Notebook 中修改参数。

- 可以在此修改 `custom_config` 的字段，再配合后续单元格写回 JSON 供 CLI 使用。


In [733]:
from copy import deepcopy
import json
from main import build_config

# Load existing notebook override (keeps __notes) when available; fall back to defaults otherwise.
if config_path.exists():
    with config_path.open("r", encoding="utf-8") as fh:
        custom_config = json.load(fh)
else:
    custom_config = deepcopy(build_config())
# custom_config
# custom_config["vehicle"]["steering_damping"]

## 关于方向盘转向回正模型说明 / Steering Assist Model
- 回正只在当前转向加速度 $|\alpha_{cmd}| \le \text{deadband}$ 时触发。(也就是说松开方向盘，角加速度为0)
- 回正加速度：$\alpha_{assist} = K_p \cdot \phi + K_d \cdot \dot{\phi}$。
- 实际转向角速度积分：$\dot{\phi}_{t+1} = \dot{\phi}_t + (\alpha_{cmd} - \alpha_{assist})\,\Delta t$。
- 方向盘角度更新：$\phi_{t+1} = \phi_t + \dot{\phi}_{t+1}\,\Delta t$。

In [734]:
# Steering tuner launch options
tuner_angle0 = 20.0   # 初始方向盘角度（度）
tuner_rate0 = 0.0    # 初始方向盘角速度（度/秒）
tuner_steps = 200     # 仿真步数
tuner_sync_updates = True  # True 表示实时写回 JSON 配置

In [735]:
from pathlib import Path
import json
import os
import subprocess

project_root = Path(".").resolve()
tuner_cmd = [
    "conda", "run", "-n", "parking-rl", "python", "steering_tuner.py",
    "--angle0", str(tuner_angle0),
    "--rate0", str(tuner_rate0),
    "--steps", str(tuner_steps),
]

if tuner_sync_updates:
    tuner_cmd.append("--sync")

if config_path is not None:
    if write_notebook_config:
        config_path.parent.mkdir(parents=True, exist_ok=True)
        with config_path.open("w", encoding="utf-8") as fh:
            json.dump(custom_config, fh, indent=2, ensure_ascii=False)
            fh.write("\n")
    tuner_cmd.extend(["--config", str(config_path.resolve())])

print("Launching steering tuner:", " ".join(tuner_cmd))
env_vars = os.environ.copy()
env_vars["MPLBACKEND"] = "QtAgg"
# subprocess.run(tuner_cmd, cwd=project_root, env=env_vars, check=False)

Launching steering tuner: conda run -n parking-rl python steering_tuner.py --angle0 20.0 --rate0 0.0 --steps 200 --sync --config /home/ansatz/ME5418/parking_project/generated_configs/notebook_override.json


In [736]:
# 调参可视化会在外部 Qt 窗口中渲染，运行上一个单元格即可启动。
custom_config = json.load(config_path.open("r", encoding="utf-8"))
print("steering_damping Kp:", custom_config["vehicle"]["steering_damping"])  # 输出当前阻尼值
print("steering_ratio Kd:", custom_config["vehicle"]["steering_rate_damping"])  # 输出当前转向比

steering_damping Kp: 48.1
steering_ratio Kd: 8.450000000000001


## 启动 CLI 演示 / Launch CLI Demo
- 生成命令并调用 `main.py`，在 Qt 窗口中运行手动或随机策略。
- 若勾选写入选项，会把上方 `custom_config` 保存成 JSON 并随命令加载。
- 请先在外部终端激活 `parking-rl` 环境，确保依赖与 GUI 后端可用。


In [737]:
from pathlib import Path
import json
import os
import subprocess

project_root = Path(".").resolve()
cmd = [
    "conda", "run", "-n", "parking-rl", "python", "main.py",
    "--mode", mode,
    "--episodes", str(episodes),
    "--max-steps", str(max_steps),
]

if sleep_scale != 0.5:
    cmd.extend(["--sleep-scale", str(sleep_scale)])

if config_path is not None:
    if write_notebook_config:
        config_path.parent.mkdir(parents=True, exist_ok=True)
        with config_path.open("w", encoding="utf-8") as fh:
            json.dump(custom_config, fh, indent=2, ensure_ascii=False)
            fh.write("\n")
    cmd.extend(["--config", str(config_path.resolve())])

print("Launching CLI demo:", " ".join(cmd))
env_vars = os.environ.copy()
env_vars["MPLBACKEND"] = "QtAgg"
subprocess.run(cmd, cwd=project_root, env=env_vars, check=False)

Launching CLI demo: conda run -n parking-rl python main.py --mode manual --episodes 1 --max-steps 4000 --config /home/ansatz/ME5418/parking_project/generated_configs/notebook_override.json
Episode 1 Step 1 Reward -13.189 Termination running Distance 7.91 Heading 129.1 deg
Episode 1 Step 2 Reward -13.196 Termination running Distance 7.91 Heading 129.1 deg
Episode 1 Step 3 Reward -13.198 Termination running Distance 7.91 Heading 129.1 deg
Episode 1 Step 4 Reward -13.203 Termination running Distance 7.91 Heading 129.1 deg
Episode 1 Step 5 Reward -13.206 Termination running Distance 7.91 Heading 129.1 deg
Episode 1 Step 6 Reward -13.208 Termination running Distance 7.91 Heading 129.1 deg
Episode 1 Step 7 Reward -13.210 Termination running Distance 7.91 Heading 129.1 deg
Episode 1 Step 8 Reward -13.208 Termination running Distance 7.92 Heading 129.1 deg
Episode 1 Step 9 Reward -13.201 Termination running Distance 7.92 Heading 129.1 deg
Episode 1 Step 10 Reward -13.208 Termination running Di

CompletedProcess(args=['conda', 'run', '-n', 'parking-rl', 'python', 'main.py', '--mode', 'manual', '--episodes', '1', '--max-steps', '4000', '--config', '/home/ansatz/ME5418/parking_project/generated_configs/notebook_override.json'], returncode=0)